## 1. 라이브러리 로드

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## 2. 대표폰트/클러스터별 폰트 로드

In [61]:
fonts = pd.read_csv('대표폰트 및 클러스터별 폰트(코사인유사도)(all).csv')

# 대표폰트 이름 기준 컬럼 정렬
column_index = fonts.columns
sorted_repres_fonts = sorted(list(fonts.loc[0]))
fonts.columns = list(fonts.loc[0])
fonts = fonts[sorted_repres_fonts]
fonts.columns = column_index
fonts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,20957738_TSTJJWanSunR_TTF,DOSIyagiBoldface,DXMgoB-KSCpc-EUC-H,EBS주시경M,HMKBP,KBIZ한마음명조 M,KoPubWorld Dotum Medium,NanumMyeongjoEcoExtraBold,Spoqa Han Sans Bold,netmarbleB,나눔손글씨 김유이체,나눔손글씨 중학생,비트로 코어 TTF,서울한강 장체M,온글잎 무궁체,온글잎 윤우체
1,20957733_전주완판본 순B_TTF,DOSGothic,10X10,BMEuljiro10yearslater,20958955_UTOIMAGE_유토이미지빨간우체통_TTF,Arita-buriB,12롯데마트드림Light,NanumGothicEco,20950867_경기천년제목TTFV_Bold,12롯데마트드림Bold,Cafe24Shiningstar,Dongle-Bold,BMDOHYEON_ttf,20950870_경기천년바탕TTF_Bold,116angmuburi,Bazzi
2,20957736_전주완판본 순L_TTF,DOSIyagiBoldface,10X10Bold,BMEULJIROTTF,BMKIRANGHAERANG-TTF,Arita-buriL,12롯데마트드림Medium,NanumGothicEcoBold,Atomy-Bold,12롯데마트행복Bold,NanumBrush,Dongle-Light,Cafe24Ssurround,20950871_경기천년바탕TTF_Regular,HMKMAMI,BinggraeSamanco-Bold
3,20957738_TSTJJWanSunR_TTF,DOSMyungjo,116watermelon,BMYEONSUNG_ttf,DXDnaraB-KSCpc-EUC-H,Arita-buriM,20956072_SuncheonR_TTF,NanumGothicEcoExtraBold,a타이틀고딕2,12롯데마트행복Light,tvN 즐거운이야기 Light,Dongle-Regular,ChangwonDangamAsac-Bold_0712,20950938_Batang_TTF,HYBDAL,BinggraeSamanco
4,20957738_전주완판본 순R_TTF,DOSPilgi,20835804_도서관체_ttf,dimibang,DXHfl2M-KSCpc-EUC-H,Arita-buriSB,Atomy-Light,NanumMyeongjoEco,a타이틀고딕3,12롯데마트행복Medium,tvN 즐거운이야기 Medium,Nanum NeuRisNeuRisCe,DXBangtango-KSCpc-EUC-H,20958935_UTOIMAGE_유토이미지고딕_R_TTF,HYBDAM,Cafe24Syongsyong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,NaN,NaN,이순신돋움체M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,중나좋체 Light,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,중나좋체 Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,한수원_한돋음_R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
repres_fonts = fonts.loc[[0]]
fonts_foreach_cluster = fonts.iloc[1:, :]
fonts_foreach_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,20957733_전주완판본 순B_TTF,DOSGothic,10X10,BMEuljiro10yearslater,20958955_UTOIMAGE_유토이미지빨간우체통_TTF,Arita-buriB,12롯데마트드림Light,NanumGothicEco,20950867_경기천년제목TTFV_Bold,12롯데마트드림Bold,Cafe24Shiningstar,Dongle-Bold,BMDOHYEON_ttf,20950870_경기천년바탕TTF_Bold,116angmuburi,Bazzi
2,20957736_전주완판본 순L_TTF,DOSIyagiBoldface,10X10Bold,BMEULJIROTTF,BMKIRANGHAERANG-TTF,Arita-buriL,12롯데마트드림Medium,NanumGothicEcoBold,Atomy-Bold,12롯데마트행복Bold,NanumBrush,Dongle-Light,Cafe24Ssurround,20950871_경기천년바탕TTF_Regular,HMKMAMI,BinggraeSamanco-Bold
3,20957738_TSTJJWanSunR_TTF,DOSMyungjo,116watermelon,BMYEONSUNG_ttf,DXDnaraB-KSCpc-EUC-H,Arita-buriM,20956072_SuncheonR_TTF,NanumGothicEcoExtraBold,a타이틀고딕2,12롯데마트행복Light,tvN 즐거운이야기 Light,Dongle-Regular,ChangwonDangamAsac-Bold_0712,20950938_Batang_TTF,HYBDAL,BinggraeSamanco
4,20957738_전주완판본 순R_TTF,DOSPilgi,20835804_도서관체_ttf,dimibang,DXHfl2M-KSCpc-EUC-H,Arita-buriSB,Atomy-Light,NanumMyeongjoEco,a타이틀고딕3,12롯데마트행복Medium,tvN 즐거운이야기 Medium,Nanum NeuRisNeuRisCe,DXBangtango-KSCpc-EUC-H,20958935_UTOIMAGE_유토이미지고딕_R_TTF,HYBDAM,Cafe24Syongsyong
5,20957740_전주완판본 각B_TTF,DOSSaemmul,20835814_SungDongGothicEB_ttf,drfont_daraehand_Basic,DXHflM-KSCpc-EUC-H,BareunBatangB,Atomy-Medium,NanumMyeongjoEcoBold,a타이틀고딕4,20956073_SuncheonB_TTF,Uhbee NaHyun,tvN 즐거운이야기 Bold,DXNPeriod-KSCpc-EUC-H,20958949_UTOIMAGE_유토이미지별나라달님_TTF,MapoDacapo,KCCImkwontaek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,NaN,NaN,이순신돋움체M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,중나좋체 Light,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,중나좋체 Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,한수원_한돋음_R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3-1. 텍스트 렌더링 함수

In [63]:
from PIL import Image, ImageDraw, ImageFont
import os

def make_image(message, rendering_time, font_list, img_save_path):

  # font setting
  font_color = 'rgb(0, 0, 0)'
  font_size = 60
  
  # image size
  width = len(message) * font_size + 10
  height = font_size + 35
  bg_color = 'rgb(255, 255, 255)'
  x_margin = 5
  y_margin = 5
  
  # 렌더링 결과 파일 저장할 폴더 생성
  img_save_path = img_save_path + message + '(' + str(rendering_time) + ')/'
  os.mkdir(img_save_path)
  
  for font in font_list:
    image =Image.new('RGB', (width, height), color = bg_color)
    font_name = font.replace('.ttf', '').replace('.TTF', '')
    font = ImageFont.truetype('C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/1. 폰트파일-_이미지파일 변환 코드/ttf_to_jpg/ttf_font(430)/' + font, font_size)
    draw = ImageDraw.Draw(image)      
    draw.text((x_margin, y_margin), message, font=font, fill=font_color)
    
    # save file
    image.save(img_save_path + font_name + '.png')
  
  return img_save_path

## 3-2. 렌더링 이미지의 공백 제거 함수

In [64]:
def trim(img):
  pixels = img.load()   
  xlist = []
  ylist = []
  for y in range(0, img.size[1]):
    for x in range(0, img.size[0]):
      if pixels[x, y] != (255, 255, 255):
        xlist.append(x)
        ylist.append(y)
  left = min(xlist)
  right = max(xlist)
  top = min(ylist)
  bottom = max(ylist)
  trimed_img = img.crop((left, top, right, bottom))
  return trimed_img

def remove_margin(img_path):
  
  if os.path.isdir(img_path):
    img_list = os.listdir(img_path)
    for img_name in img_list:
      img = Image.open(img_path + img_name)
      trimed_img = trim(img)
      trimed_img.save(img_path + img_name)
      
  else:
    img = Image.open(img_path) #.convert('RGB')
    trimed_img = trim(img)
    trimed_img.save(img_path)
    

## 4. 입력이미지 특징벡터 추출 

In [65]:
test_img_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/116angmuburi.png'
remove_margin(test_img_path)

In [66]:
import sys
sys.path.append("C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/3. 유사폰트추천코드/img2vec/img2vec_pytorch")  # Adds higher directory to python modules path.
from img_to_vec import Img2Vec

img2vec =Img2Vec(model="inception")

test_img_input_path = test_img_path
test_filename = os.fsdecode(test_img_input_path)
test_img = Image.open(os.path.join(test_img_input_path, test_filename))
test_img_vector = img2vec.get_vec(test_img)
    
test_img_vector_df = pd.DataFrame(test_img_vector).transpose()
test_img_vector_df

AdaptiveAvgPool2d(output_size=(1, 1))


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.620054,0.009488,0.012001,0.0,0.059348,0.002545,0.013653,0.973282,0.000644,0.348316,...,0.806429,1.290198,0.0,0.139708,0.8056,0.031797,0.703384,0.003193,0.150967,1.064167


## 5. 1차 유사도 비교

### 5.1 대표폰트로 텍스트 렌더링

In [67]:
font_list = os.listdir('C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/1. 폰트파일-_이미지파일 변환 코드/ttf_to_jpg/ttf_font(430)')

repres_font_list = []
for font_idx in repres_fonts:
  repres_font_list.append(repres_fonts[font_idx].values[0] + '.ttf')

msg = "아자아자"
rendering_time = 1
img_save_path = "C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/"
rendered_img_path = make_image(msg, rendering_time, repres_font_list, img_save_path)

remove_margin(rendered_img_path)


### 5.2 렌더링한 텍스트 이미지의 특징벡터 추출

In [ ]:
input_path = rendered_img_path
list_pics1 = []
filenames1 = []
for file1 in os.listdir(input_path):
  filename1 = os.fsdecode(file1)
  img1 = Image.open(os.path.join(input_path, filename1)).convert('RGB')
  list_pics1.append(img1)
  filenames1.append(filename1)
    
vectors1 = img2vec.get_vec(list_pics1)
pics1 = {}
for i1, vec1 in enumerate(vectors1):
  pics1[filenames1[i1]] = vec1
    
rendering_1st_vector_df = pd.DataFrame(pics1).transpose()
# rendering_1st_vector_df

### 5.3 입력이미지 - 대표폰트 렌더링 이미지의 유사도 비교

In [69]:
rendering_1st_vec_sim = []
for i in range(len(rendering_1st_vector_df)):
  rendering_1st_vec_sim.append(cosine_similarity(test_img_vector_df.values, rendering_1st_vector_df.iloc[[i]].values)[0][0])

rendering_1st_vector_sim_df = rendering_1st_vector_df
rendering_1st_vector_sim_df['similarity'] = rendering_1st_vec_sim

### 5.4 대표폰트 중 입력이미지와 가장 유사한 폰트 도출

In [70]:
rendering_1st_vector_sim_df.reset_index(drop=False, inplace=True)
rendering_1st_vector_sim_df
most_sim_font_1st = rendering_1st_vector_sim_df.iloc[[rendering_1st_vector_df['similarity'].argmax()]].index[0]

In [86]:
print("대표폰트 중 입력이미지와 가장 유사한 폰트 : ", most_sim_font_1st)
# rendering_1st_vector_sim_df

대표폰트 중 입력이미지와 가장 유사한 폰트 :  5


## 6. 2차 렌더링

### 6.1 가장 유사한 대표폰트가 해당된 클러스터의 폰트로 렌더링

In [72]:
font_list_2nd_rendering = fonts_foreach_cluster.iloc[:, most_sim_font_1st].dropna().to_numpy()

most_sim_cluster_font_list = []
for font in font_list_2nd_rendering:
  most_sim_cluster_font_list.append(font + '.ttf')

rendering_time = 2
rendered_img_path = make_image(msg, rendering_time, most_sim_cluster_font_list, img_save_path)

remove_margin(rendered_img_path)

### 6.2 렌더링한 텍스트 이미지의 특징벡터 추출

In [83]:
input_path = rendered_img_path
list_pics2 = []
filenames2 = []
for file2 in os.listdir(input_path):
    filename2 = os.fsdecode(file2)
    img2 = Image.open(os.path.join(input_path, filename2)).convert('RGB')
    list_pics2.append(img2)
    filenames2.append(filename2)
    
vectors2 = img2vec.get_vec(list_pics2)
pics2 = {}
for i2, vec2 in enumerate(vectors2):
    pics2[filenames2[i2]] = vec2
    
rendering_2st_vector_df = pd.DataFrame(pics2).transpose()
# rendering_2st_vector_df

(37, 2048)


### 6.3 입력이미지 - 클러스터폰트 렌더링 이미지의 유사도 비교

In [74]:
rendering_2st_vec_sim = []
for i in range(len(rendering_2st_vector_df)):
  rendering_2st_vec_sim.append(cosine_similarity(test_img_vector_df.values, rendering_2st_vector_df.iloc[[i]].values)[0][0])

rendering_2st_vector_sim_df = rendering_2st_vector_df
rendering_2st_vector_sim_df['similarity'] = rendering_2st_vec_sim

### 6.4 입력이미지와 가장 유사한 폰트 및 유사 폰트 추천 순위 도출

In [82]:
most_sim_font_2st = rendering_2st_vector_sim_df.iloc[[rendering_2st_vector_df['similarity'].argmax()]]
print('입력이미지와 가장 유사한 폰트 : ', most_sim_font_2st.index[0].replace('.png', ''))

### 추천 폰트 순위
rendering_2st_vector_sim_df.sort_values(by=['similarity'],ascending=False)

rendering_2st_vector_sim_df[['similarity']].sort_values(by=['similarity'],ascending=False)

입력이미지와 가장 유사한 폰트 :  나눔명조


,similarity
나눔명조.png,0.770239
Arita-buriL.png,0.770205
BareunBatangL.png,0.751095
조선일보명조.png,0.735097
JejuMyeongjo.png,0.730613
KoPubWorld Batang Light.png,0.730159
H2MJSM.png,0.728505
HY신명조.png,0.728505
KBIZ한마음명조 R.png,0.727915
HMKMM.png,0.721063
